### References:

Darmet, L., Ladouce, S., & Dehais, F. (2023). **Shortened calibration of code-VEP based BCI by improved deep learning 
architecture and golden subjects pre-training.** *In 11th International IEEE EMBS Conference on Neural Engineering.* 

Source: https://github.com/neuroergoISAE/offline_cVEP_bitwise

Thielen, J., Marsman, P., Farquhar, J., & Desain, P. (2021). **From full calibration to zero training for a code-modulated
visual evoked potentials for brain–computer interface**. *Journal of Neural Engineering, 18(5), 056007.*

Source: https://gitlab.socsci.ru.nl/jthielen/pynt/-/blob/main/examples

Sebastian Nagel

Source: https://github.com/thecodingdad/eeg2code

Teulings, D.E.J. David (2022).  **an-application-of-recurrency-to-eeg2code-deep-learning**

Source: https://gitlab.socsci.ru.nl/D.Teulings/an-application-of-recurrency-to-eeg2code-deep-learning/-/tree/master



In [1]:
# the path to the dataset
path = 'D:/Dataset'

In [2]:
import os
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import pynt
import keras
import sys
import scipy.io as sio
import tensorflow as tf
import time
import seaborn
import random
import keras.utils as ku
from scipy import stats
from scipy.stats import wilcoxon
from keras.optimizers import adam
from keras.callbacks import History 
from sklearn.metrics import accuracy_score
from tensorflow.python.keras import backend
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras import initializers
from keras.models import load_model
from keras.models import Sequential
from keras.layers import (Input,
                          InputLayer,
                          GRU,
                          RNN,
                          LSTM,
                          Reshape,
                          SimpleRNN,
                          Conv1D,
                          Conv2D,
                          MaxPooling1D,
                          MaxPooling2D,
                          Permute,
                          Flatten, 
                          Dense,
                          BatchNormalization, 
                          Activation,
                          Dropout
                         )
from scipy.io import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cross_decomposition import CCA
from tensorflow.python.keras import backend
from tensorflow.keras.models import load_model

# Settings
### Hyperparameters, function that load data, and function that slicing data to epoch

In [3]:
"""
Hyperparameters.

subjects = ['sub-01','sub-02', 'sub-03', 'sub-04', 'sub-05','sub-06','sub-07', 'sub-08', 'sub-09', 'sub-10'
           ,'sub-11','sub-22', 'sub-13', 'sub-14', 'sub-15','sub-16','sub-17', 'sub-18', 'sub-19', 'sub-20'
           ,'sub-21','sub-22', 'sub-23', 'sub-24', 'sub-25','sub-26','sub-27', 'sub-28', 'sub-29', 'sub-30']
epoch_size: 0.3 * 480 sampling frequency = 144 window size
step_size: 1 / 60 frame rate * 480 sampling frequency = 8
learning rate: 0.001 same as EEG2Code paper
batch size: 256 same as EEG2Code paper
    Mini-batch gradient descent
epochs: 25 same as EE2Code paper
    Number of iterations
Optimizers: Adam
    adam = extension of stochastic gradient descent to update network weights during training.
    Updates the learning rate for each network weight individually.
n_folds: 5 k-fold cross validation
"""
subjects =  ['sub-01','sub-02', 'sub-03', 'sub-04', 'sub-05','sub-06','sub-07', 'sub-08', 'sub-09', 'sub-10']
epoch_size = int(480 * 0.3)
step_size = int(1 / 60 * 480)
learning_rate = 0.001
batch_size = 256
epochs = 50
adam = keras.optimizers.Adam(learning_rate=learning_rate, amsgrad=True)
n_folds = 5

#Keras uses TensorFlow and utilize CPU only. (Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz 4.20 GHz)
#30 Participants, 256 batch size, 25 epchs, 5 fold takes 40+ hours to train and test EEG2Code model.
#For training and test LDA and CCA+LDA takes 10+ minutes.

In [4]:
"""
Pre-processed settings

band_pass: [2.0 Hz, 30.0 Hz]
    Bandpass cutoffs in Hz for spectral filtering
tmin: 0.0 sec
    Trial onset in seconds for slicing
tmax: 31.5 sec
    Trial end in seconds for slicing
    
Load data.

X: (100 trials, 8 channels, 15120 samples)
    EEG time-series data
V: (1008 samples, 20 classes/labels/symbols)
    Codes matrix. Codebook
y: (100 trials,)
    Label vector of 100 trials
fs = 480 Hz
"""
def load_data(subject):
    fn = os.path.join(path, 'derivatives480', 'offline', subject, f'{subject}_gdf.npz')
    tmp = np.load(fn)
    X = tmp['X']
    V = tmp['V']
    y = tmp['y']
    fs = tmp['fs']
    return X, V, y, fs

In [5]:
"""
Slices trials to epochs.

Returns:
    X_sliced: (100 trials, 1872 epochs, 8 channels, 144 window size)
        The sliced EEG data of shape with epoch
    y_sliced: (100 trials, 1872 epochs)
        The sliced label information of shape with epoch
"""
def trials_to_epochs(X, y, codes, epoch_size, step_size):
    n_trials, n_channels, n_samples = X.shape
    n_epochs = int((n_samples - epoch_size) / step_size)
    X_sliced = np.zeros((n_trials, n_epochs, n_channels, epoch_size), 
        dtype='float32')
    y_sliced = np.zeros((n_trials, n_epochs), dtype='uint8')
    for i_epoch in range(n_epochs):
        start = i_epoch * step_size
        X_sliced[:, i_epoch, :, :] = X[:, :, start:start+epoch_size]
        y_sliced[:, i_epoch] = codes[start % codes.shape[0], y]
    return X_sliced, y_sliced

# Method
### EEG2Code

In [6]:
"""
EEG2Code Architecture

Difference from paper
    Maxpooling 1D(1 pool size , 1 strides) -> Maxpooling2D((1, 1), (1, 1))
    Conv1d(kernel size) -> Conv2d((1,kernel size))
    Dense(1024) -> Dense(256)
    Softmax -> Sigmoid
"""
def EEG2Code(n_channels, epoch_size):
    model = Sequential()
    # layer 1
    model.add(Conv2D(16, input_shape=(n_channels, epoch_size, 1),
        kernel_size=(n_channels, 1),
        padding='valid',
        strides=(1, 1)))
    model.add(BatchNormalization(axis=3, scale=False, center=False))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(
        pool_size=(1, 1), # 2,1
        padding='same', 
        strides=(1, 1))) 
    # layer 2
    model.add(Conv2D(8, 
        kernel_size=(1, 64),
        padding='same', 
        strides=(1, 1)))
    model.add(BatchNormalization(axis=3, scale=False, center=False))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(
        pool_size=(1, 1), # 2,1
        padding='same', 
        strides=(1, 1))) 
    model.add(Dropout(0.5))
    # layer 3
    model.add(Conv2D(4,
        kernel_size=(1, 5),
        padding='same',
        strides=(1, 1)))
    model.add(BatchNormalization(axis=3, scale=False, center=False))               
    model.add(Activation('relu'))
    model.add(MaxPooling2D(
        pool_size=(1, 1), # 2,1
        padding='same', 
        strides=(1, 1)))
    model.add(Dropout(0.5))
    # layer 4
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    # layer 5
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [7]:
"""
Train, test, evaluate EEG2Code
X_sliced: (100 trials, 1872 epochs, 8 channels, 144 window size)
X_sliced_reshape: (187200 trials*epochs, 8 channels, 144 window size, 1)
y_sliced: (100 trials, 1872 epochs)
y_sliced_reshape: (187200 trials*epochs,)
_V: (20 symbols, 1872 epochs)
"""
# Load data only for n_trials, n_channels, n_samples
X1, V1, y1, fs1 = load_data(subjects[0])
n_trials, n_channels, n_samples = X1.shape

# Pre-defined sequence of indices for fold assignments
folds_ntrial = np.repeat(np.arange(n_folds), n_trials / n_folds)
folds_ntrial_epochs = np.repeat(np.arange(n_folds), int(n_trials*(n_samples - epoch_size) / step_size / n_folds))

# Store accuracies for each subjects
EEG2Code_acc_epoch = []
EEG2Code_acc_trial = []

# Store epoch predictions of all fold of all subjects
epoch_predictions_saveall_eeg2code = np.empty((len(subjects), n_folds, int((n_samples - epoch_size) / step_size * (n_trials / n_folds))))

#Loop over all subjects
for j, subject in enumerate(subjects):
    # Load data
    X, V, y, fs = load_data(subject)
    print('Subject: ', subject)
    
    # Set up dataset for epoch classification
    X_sliced, y_sliced = trials_to_epochs(X, y, V, epoch_size, step_size)
    X_sliced_reshape = X_sliced.reshape(-1, X_sliced.shape[2], X_sliced.shape[3], 1)
    y_sliced_reshape = y_sliced.flatten()
    
    # Set up codebook for trial classification
    n = int(np.ceil(n_samples / V.shape[0]))    
    _V = np.tile(V, (n, 1)).astype('float32')[:n_samples-epoch_size:step_size]
    _V = _V.transpose(1,0)

    # Store accuracies for each fold
    accuracy_epoch_eeg2code = np.zeros(n_folds)
    accuracy_trial_eeg2code = np.zeros(n_folds)
    
    # Store epoch predictions of all folds
    epoch_predictions_save_eeg2code= np.empty((n_folds,int((n_samples - epoch_size) / step_size *n_trials/n_folds)))

    # Loop over 5-fold cross validation
    for i_fold in range(n_folds):
        # Split data to train and valid set
        X_train = X_sliced_reshape[folds_ntrial_epochs  != i_fold,:]
        X_test1 = X_sliced_reshape[folds_ntrial_epochs  == i_fold,:]
        y_train = y_sliced_reshape[folds_ntrial_epochs  != i_fold]
        y_test = y_sliced_reshape[folds_ntrial_epochs  == i_fold]
        test_y = y[folds_ntrial  == i_fold]
        
        # Create a new instance of EEG2Code model
        n_channels = X_train.shape[1]
        eeg2code = EEG2Code(n_channels, epoch_size)
        eeg2code.compile(
            loss=keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer=adam,
            
            metrics=['accuracy'])
        #Print model structure
        if(subject == 'sub-01' and i_fold == 0):
            for i in eeg2code.layers:
                print(i.name, "(', i.__class__.__name__, ')")
                print('     Input shape : ',i.input_shape)
                print('     Output shape: ', i.output_shape)
                print('     Trainable Weights(Param #): ', i.count_params())
                
        # Split data into training and validation set
        X_tr1, X_val1, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=None)
        
        # Normalize X training data and apply same parameter to validation and test dataset
        norm_mean = np.mean(X_tr1)
        norm_std = np.std(X_tr1)
        X_tr = (X_tr1 - norm_mean) / norm_std
        X_val = (X_val1 - norm_mean) / norm_std
        X_test = (X_test1 - norm_mean) / norm_std

        # Normalize training data and train model
        history = eeg2code.fit(X_tr, y_tr, batch_size=batch_size, epochs=epochs, 
                               validation_data = (X_val, y_val), shuffle=True,verbose=2)
        
        # Save training history
        hist_df = pd.DataFrame(history.history)
        if not os.path.exists(os.path.join(path, 'history1')):
            os.makedirs(os.path.join(path, 'history1'))
        with open(os.path.join(path, 'history1', f'{subject}_fold_{i_fold+1}_EEG2Codehistory.json'), 'w') as f:
             hist_df.to_json(f)
        
        # Evaluate accuracy on epoch level (predict epoch probability)
        epoch_predictions = eeg2code.predict(X_test)
        epoch_threshhold = (epoch_predictions > 0.5).astype(int)
        accuracy_epoch_eeg2code[i_fold] = 100*np.mean(y_test == epoch_threshhold.flatten())
        
        # Append epoch predictions for fold
        epoch_predictions_save_eeg2code[i_fold] = np.array(epoch_predictions.flatten())
        
        # Evaluate accuracy on trial level (predict sequence)
        trial_predictions = epoch_predictions.reshape(20,1872)
        # Use correlation to match predicted pattern to specific symbol
        predictions = np.zeros(20)
        for i in range(20):
            rho = np.corrcoef(trial_predictions[i,:],_V)[0, 1:]
            predictions[i]=np.argmax(rho)
        sum_value = sum(predictions == test_y)
        trial_score = sum_value/20*100
        accuracy_trial_eeg2code[i_fold] = trial_score
        
        print(f"fold: {i_fold+1} is complete")
        
    # Append the averaged fold accuracies for the subject
    EEG2Code_acc_epoch.append(accuracy_epoch_eeg2code.mean())
    EEG2Code_acc_trial.append(accuracy_trial_eeg2code.mean())
                                      
    # Append epoch predictions of all folds for subject
    epoch_predictions_saveall_eeg2code[j] = epoch_predictions_save_eeg2code



Subject:  sub-01
conv2d (', i.__class__.__name__, ')
     Input shape :  (None, 8, 144, 1)
     Output shape:  (None, 1, 144, 16)
     Trainable Weights(Param #):  144
batch_normalization (', i.__class__.__name__, ')
     Input shape :  (None, 1, 144, 16)
     Output shape:  (None, 1, 144, 16)
     Trainable Weights(Param #):  32
activation (', i.__class__.__name__, ')
     Input shape :  (None, 1, 144, 16)
     Output shape:  (None, 1, 144, 16)
     Trainable Weights(Param #):  0
max_pooling2d (', i.__class__.__name__, ')
     Input shape :  (None, 1, 144, 16)
     Output shape:  (None, 1, 144, 16)
     Trainable Weights(Param #):  0
conv2d_1 (', i.__class__.__name__, ')
     Input shape :  (None, 1, 144, 16)
     Output shape:  (None, 1, 144, 8)
     Trainable Weights(Param #):  8200
batch_normalization_1 (', i.__class__.__name__, ')
     Input shape :  (None, 1, 144, 8)
     Output shape:  (None, 1, 144, 8)
     Trainable Weights(Param #):  16
activation_1 (', i.__class__.__name__, 

Epoch 42/50
468/468 - 68s - loss: 0.4783 - accuracy: 0.7771 - val_loss: 0.4406 - val_accuracy: 0.8035 - 68s/epoch - 146ms/step
Epoch 43/50
468/468 - 69s - loss: 0.4776 - accuracy: 0.7764 - val_loss: 0.4410 - val_accuracy: 0.8036 - 69s/epoch - 146ms/step
Epoch 44/50
468/468 - 68s - loss: 0.4770 - accuracy: 0.7763 - val_loss: 0.4387 - val_accuracy: 0.8037 - 68s/epoch - 145ms/step
Epoch 45/50
468/468 - 68s - loss: 0.4783 - accuracy: 0.7755 - val_loss: 0.4437 - val_accuracy: 0.8025 - 68s/epoch - 146ms/step
Epoch 46/50
468/468 - 68s - loss: 0.4778 - accuracy: 0.7765 - val_loss: 0.4409 - val_accuracy: 0.8039 - 68s/epoch - 145ms/step
Epoch 47/50
468/468 - 68s - loss: 0.4766 - accuracy: 0.7765 - val_loss: 0.4388 - val_accuracy: 0.8064 - 68s/epoch - 145ms/step
Epoch 48/50
468/468 - 68s - loss: 0.4783 - accuracy: 0.7761 - val_loss: 0.4392 - val_accuracy: 0.8046 - 68s/epoch - 146ms/step
Epoch 49/50
468/468 - 68s - loss: 0.4757 - accuracy: 0.7774 - val_loss: 0.4394 - val_accuracy: 0.8046 - 68s/epo

Epoch 6/50
468/468 - 63s - loss: 0.5358 - accuracy: 0.7372 - val_loss: 0.4876 - val_accuracy: 0.7703 - 63s/epoch - 134ms/step
Epoch 7/50
468/468 - 63s - loss: 0.5284 - accuracy: 0.7438 - val_loss: 0.4796 - val_accuracy: 0.7732 - 63s/epoch - 135ms/step
Epoch 8/50
468/468 - 64s - loss: 0.5249 - accuracy: 0.7451 - val_loss: 0.4815 - val_accuracy: 0.7749 - 64s/epoch - 137ms/step
Epoch 9/50
468/468 - 64s - loss: 0.5222 - accuracy: 0.7476 - val_loss: 0.4784 - val_accuracy: 0.7760 - 64s/epoch - 137ms/step
Epoch 10/50
468/468 - 63s - loss: 0.5190 - accuracy: 0.7499 - val_loss: 0.4740 - val_accuracy: 0.7788 - 63s/epoch - 135ms/step
Epoch 11/50
468/468 - 63s - loss: 0.5165 - accuracy: 0.7535 - val_loss: 0.4724 - val_accuracy: 0.7803 - 63s/epoch - 134ms/step
Epoch 12/50
468/468 - 63s - loss: 0.5126 - accuracy: 0.7550 - val_loss: 0.4712 - val_accuracy: 0.7805 - 63s/epoch - 135ms/step
Epoch 13/50
468/468 - 63s - loss: 0.5114 - accuracy: 0.7565 - val_loss: 0.4663 - val_accuracy: 0.7848 - 63s/epoch -

468/468 - 65s - loss: 0.4972 - accuracy: 0.7648 - val_loss: 0.4596 - val_accuracy: 0.7940 - 65s/epoch - 139ms/step
Epoch 21/50
468/468 - 65s - loss: 0.4959 - accuracy: 0.7667 - val_loss: 0.4587 - val_accuracy: 0.7947 - 65s/epoch - 139ms/step
Epoch 22/50
468/468 - 65s - loss: 0.4950 - accuracy: 0.7672 - val_loss: 0.4553 - val_accuracy: 0.7943 - 65s/epoch - 139ms/step
Epoch 23/50
468/468 - 65s - loss: 0.4956 - accuracy: 0.7669 - val_loss: 0.4557 - val_accuracy: 0.7947 - 65s/epoch - 139ms/step
Epoch 24/50
468/468 - 65s - loss: 0.4939 - accuracy: 0.7669 - val_loss: 0.4552 - val_accuracy: 0.7968 - 65s/epoch - 140ms/step
Epoch 25/50
468/468 - 58s - loss: 0.4940 - accuracy: 0.7671 - val_loss: 0.4536 - val_accuracy: 0.7976 - 58s/epoch - 124ms/step
Epoch 26/50
468/468 - 65s - loss: 0.4926 - accuracy: 0.7690 - val_loss: 0.4526 - val_accuracy: 0.7964 - 65s/epoch - 139ms/step
Epoch 27/50
468/468 - 66s - loss: 0.4933 - accuracy: 0.7679 - val_loss: 0.4565 - val_accuracy: 0.7958 - 66s/epoch - 140ms/s

468/468 - 64s - loss: 0.4803 - accuracy: 0.7773 - val_loss: 0.4395 - val_accuracy: 0.8022 - 64s/epoch - 137ms/step
Epoch 35/50
468/468 - 64s - loss: 0.4821 - accuracy: 0.7750 - val_loss: 0.4424 - val_accuracy: 0.8011 - 64s/epoch - 137ms/step
Epoch 36/50
468/468 - 65s - loss: 0.4802 - accuracy: 0.7768 - val_loss: 0.4398 - val_accuracy: 0.8008 - 65s/epoch - 139ms/step
Epoch 37/50
468/468 - 64s - loss: 0.4824 - accuracy: 0.7752 - val_loss: 0.4440 - val_accuracy: 0.8018 - 64s/epoch - 137ms/step
Epoch 38/50
468/468 - 64s - loss: 0.4804 - accuracy: 0.7760 - val_loss: 0.4369 - val_accuracy: 0.8039 - 64s/epoch - 137ms/step
Epoch 39/50
468/468 - 64s - loss: 0.4815 - accuracy: 0.7766 - val_loss: 0.4386 - val_accuracy: 0.8049 - 64s/epoch - 138ms/step
Epoch 40/50
468/468 - 64s - loss: 0.4802 - accuracy: 0.7762 - val_loss: 0.4387 - val_accuracy: 0.8047 - 64s/epoch - 138ms/step
Epoch 41/50
468/468 - 64s - loss: 0.4803 - accuracy: 0.7769 - val_loss: 0.4367 - val_accuracy: 0.8038 - 64s/epoch - 137ms/s

...

Epoch 46/50
468/468 - 39s - loss: 0.6931 - accuracy: 0.5013 - val_loss: 0.6932 - val_accuracy: 0.5008 - 39s/epoch - 84ms/step
Epoch 47/50
468/468 - 41s - loss: 0.6932 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.4968 - 41s/epoch - 87ms/step
Epoch 48/50
468/468 - 41s - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6932 - val_accuracy: 0.4967 - 41s/epoch - 87ms/step
Epoch 49/50
468/468 - 41s - loss: 0.6932 - accuracy: 0.4980 - val_loss: 0.6932 - val_accuracy: 0.4971 - 41s/epoch - 87ms/step
Epoch 50/50
468/468 - 40s - loss: 0.6932 - accuracy: 0.4987 - val_loss: 0.6932 - val_accuracy: 0.4984 - 40s/epoch - 86ms/step
1170/1170 [==============================] - 4s 3ms/step
fold: 1 is complete
Epoch 1/50
468/468 - 41s - loss: 0.7007 - accuracy: 0.4990 - val_loss: 0.6932 - val_accuracy: 0.4984 - 41s/epoch - 89ms/step
Epoch 2/50
468/468 - 41s - loss: 0.6933 - accuracy: 0.4989 - val_loss: 0.6932 - val_accuracy: 0.4982 - 41s/epoch - 87ms/step
Epoch 3/50
468/468 - 39s - loss: 0.6933 - a

C:\Users\Sun Geun Yu\anaconda3\lib\site-packages\numpy\lib\function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Sun Geun Yu\anaconda3\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


fold: 2 is complete
Epoch 1/50
468/468 - 39s - loss: 0.6990 - accuracy: 0.4987 - val_loss: 0.6931 - val_accuracy: 0.5021 - 39s/epoch - 84ms/step
Epoch 2/50
468/468 - 39s - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.4989 - 39s/epoch - 82ms/step
Epoch 3/50
468/468 - 39s - loss: 0.6932 - accuracy: 0.4991 - val_loss: 0.6932 - val_accuracy: 0.4990 - 39s/epoch - 82ms/step
Epoch 4/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.5028 - val_loss: 0.6932 - val_accuracy: 0.4988 - 38s/epoch - 82ms/step
Epoch 5/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.5011 - val_loss: 0.6932 - val_accuracy: 0.4984 - 38s/epoch - 82ms/step
Epoch 6/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.4969 - val_loss: 0.6932 - val_accuracy: 0.4987 - 38s/epoch - 82ms/step
Epoch 7/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.5009 - val_loss: 0.6932 - val_accuracy: 0.5011 - 38s/epoch - 82ms/step
Epoch 8/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.4993 - 38

Epoch 16/50
468/468 - 41s - loss: 0.6932 - accuracy: 0.4982 - val_loss: 0.6932 - val_accuracy: 0.4975 - 41s/epoch - 88ms/step
Epoch 17/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.4996 - val_loss: 0.6932 - val_accuracy: 0.5019 - 38s/epoch - 82ms/step
Epoch 18/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.4974 - 38s/epoch - 82ms/step
Epoch 19/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6932 - val_accuracy: 0.4964 - 38s/epoch - 82ms/step
Epoch 20/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.5019 - val_loss: 0.6932 - val_accuracy: 0.5014 - 38s/epoch - 82ms/step
Epoch 21/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.4995 - val_loss: 0.6931 - val_accuracy: 0.5016 - 38s/epoch - 82ms/step
Epoch 22/50
468/468 - 38s - loss: 0.6932 - accuracy: 0.4977 - val_loss: 0.6932 - val_accuracy: 0.4973 - 38s/epoch - 82ms/step
Epoch 23/50
468/468 - 39s - loss: 0.6932 - accuracy: 0.5014 - val_loss: 0.6932 - val_accuracy: 0.5015 - 39s/epoch - 82

Epoch 31/50
468/468 - 40s - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5004 - 40s/epoch - 86ms/step
Epoch 32/50
468/468 - 40s - loss: 0.6931 - accuracy: 0.5002 - val_loss: 0.6932 - val_accuracy: 0.5017 - 40s/epoch - 86ms/step
Epoch 33/50
468/468 - 40s - loss: 0.6932 - accuracy: 0.4999 - val_loss: 0.6932 - val_accuracy: 0.5000 - 40s/epoch - 86ms/step
Epoch 34/50
468/468 - 40s - loss: 0.6931 - accuracy: 0.4995 - val_loss: 0.6932 - val_accuracy: 0.4989 - 40s/epoch - 86ms/step
Epoch 35/50
468/468 - 40s - loss: 0.6931 - accuracy: 0.4995 - val_loss: 0.6932 - val_accuracy: 0.5025 - 40s/epoch - 86ms/step
Epoch 36/50
468/468 - 40s - loss: 0.6931 - accuracy: 0.5023 - val_loss: 0.6932 - val_accuracy: 0.5013 - 40s/epoch - 86ms/step
Epoch 37/50
468/468 - 40s - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6932 - val_accuracy: 0.4987 - 40s/epoch - 86ms/step
Epoch 38/50
468/468 - 40s - loss: 0.6931 - accuracy: 0.4982 - val_loss: 0.6932 - val_accuracy: 0.4992 - 40s/epoch - 86

## Save data

In [8]:
"""
Save training history and epoch level classification
"""
# Save accuracies history
with open(os.path.join(path, 'history1', 'EEG2Code_epoch_accuracies.txt'), 'w') as f:
    for acc in EEG2Code_acc_epoch:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history1', 'EEG2Code_trial_accuracies.txt'), 'w') as f:
    for acc in EEG2Code_acc_trial:
        f.write('%s\n' % acc)
        
# Save epoch predictions of all fold of all subjects
if not os.path.exists(os.path.join(path, 'history1')):
    os.makedirs(os.path.join(path, 'history1'))
np.save(os.path.join(path, 'history1', 'epoch_predictions_saveall_eeg2code.npy'), epoch_predictions_saveall_eeg2code)

### LDA, CCA+LDA

In [9]:
# Store accuracies for each subjects
LDA_acc_epoch = []
LDA_acc_trial = []
CCA_acc_epoch = []
CCA_acc_trial = []

# Store accuracies for each subjects

epoch_predictions_saveall_lda = np.empty((len(subjects),n_folds,int(n_trials/n_folds*(n_samples - epoch_size) / step_size)))
epoch_predictions_saveall_cca = np.empty((len(subjects),n_folds,int(n_trials/n_folds*(n_samples - epoch_size) / step_size)))

# Loop over all subjects
for i, subject in enumerate(subjects):
    X, V, y, fs = load_data(subject)
    print('Subject: ', subject)
    n_trials, n_channels, n_samples = X.shape
    # Set trial duration

    # Setup cross-validation
    folds = np.repeat(np.arange(n_folds), n_trials / n_folds)

    # Set up codebook for trial classification
    n = int(np.ceil(n_samples / V.shape[0]))
    _V = np.tile(V, (n, 1)).astype('float32')[:n_samples-epoch_size:step_size]

    # Setup LDAs
    lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
    lda2 = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
    
    # Setup CCA
    cca = CCA(n_components=1)
    
    # Loop folds
    accuracy_epoch_lda = np.zeros(n_folds)
    accuracy_trial_lda = np.zeros(n_folds)
    accuracy_epoch_cca = np.zeros(n_folds)
    accuracy_trial_cca = np.zeros(n_folds)
    
    # Store epoch predictions of all folds
    epoch_predictions_save_lda = np.empty((n_folds,int((n_samples - epoch_size) / step_size *n_trials/n_folds)))
    epoch_predictions_save_cca = np.empty((n_folds,int((n_samples - epoch_size) / step_size *n_trials/n_folds)))
    
    for i_fold in range(n_folds):

        # Split data to train and valid set
        X_trn, y_trn = X[folds != i_fold, :, :n_samples], y[folds != i_fold]
        X_tst, y_tst = X[folds == i_fold, :, :n_samples], y[folds == i_fold]
        
        # Slice trials to epochs
        X_sliced_trn, y_sliced_trn = trials_to_epochs(X_trn, y_trn, V, epoch_size, step_size)
        X_sliced_tst, y_sliced_tst = trials_to_epochs(X_tst, y_tst, V, epoch_size, step_size)
        
        """
        LDA method
        """
        
        # Train LDA (on epoch level)
        # N.B.: spatio-temporal features are flattened
        # N.B.: all epochs of all trials are concatenated
        lda.fit(X_sliced_trn.reshape((-1, n_channels * epoch_size)), y_sliced_trn.flatten())

        # Apply LDA (on epoch level)
        yh_sliced_tst_lda = lda.predict(X_sliced_tst.reshape((-1, n_channels * epoch_size)))
        
        # Append LDA epoch predictions for fold
        epoch_predictions_save_lda[i_fold] = np.array(yh_sliced_tst_lda.flatten())
        
        # Compute accuracy (on epoch level)
        accuracy_epoch_lda[i_fold] = 100 * np.mean(yh_sliced_tst_lda == y_sliced_tst.flatten())

        # Apply LDA (on trial level)
        ph_tst_lda = lda.predict_proba(X_sliced_tst.reshape((-1, n_channels * epoch_size)))[:, 1]
        ph_tst_lda = np.reshape(ph_tst_lda, y_sliced_tst.shape)
        rho_lda = pynt.utilities.correlation(ph_tst_lda, _V.T)
        yh_tst_lda = np.argmax(rho_lda, axis=1)
        accuracy_trial_lda[i_fold] = 100 * np.mean(yh_tst_lda == y_tst)
        
        """
        CCA+ LDA method
        """
        
        # Train CCA (on epoch level)
        erp_noflash = np.mean(X_sliced_trn[y_sliced_trn == 0, :, :], axis=0, keepdims=True)
        erp_flash = np.mean(X_sliced_trn[y_sliced_trn == 1, :, :], axis=0, keepdims=True)
        erps = np.concatenate((erp_noflash, erp_flash), axis=0)[y_sliced_trn, :, :]
        cca.fit(
            X_sliced_trn.transpose((0, 1, 3, 2)).reshape((-1, n_channels)), 
            erps.transpose((0, 1, 3, 2)).reshape(-1, n_channels))
        w = cca.x_weights_.flatten()
        
        # Apply CCA (on epoch level)
        X_sliced_filtered_trn = np.dot(w, 
            X_sliced_trn.transpose((2, 0, 1, 3)).reshape((n_channels, -1))).reshape(-1, epoch_size)
        X_sliced_filtered_tst = np.dot(w, 
            X_sliced_tst.transpose((2, 0, 1, 3)).reshape((n_channels, -1))).reshape(-1, epoch_size)
        
        # Train LDA (on epoch level)
        # N.B.: spatio-temporal features are flattened
        # N.B.: all epochs of all trials are concatenated
        lda2.fit(X_sliced_filtered_trn, y_sliced_trn.flatten())

        # Apply LDA (on epoch level)
        yh_sliced_tst_cca = lda2.predict(X_sliced_filtered_tst)
                                          
        # Append CCA+LDA epoch predictions for fold
        epoch_predictions_save_cca[i_fold] = np.array(yh_sliced_tst_cca.flatten())
                                          
        # Compute accuracy (on epoch level)
        accuracy_epoch_cca[i_fold] = 100 * np.mean(yh_sliced_tst_cca == y_sliced_tst.flatten())

        # Apply LDA (on trial level)
        ph_tst_cca = lda2.predict_proba(X_sliced_filtered_tst)[:, 1]
        ph_tst_cca = np.reshape(ph_tst_cca, y_sliced_tst.shape)
        rho_cca = pynt.utilities.correlation(ph_tst_cca, _V.T)
        yh_tst_cca = np.argmax(rho_cca, axis=1)
        accuracy_trial_cca[i_fold] = 100 * np.mean(yh_tst_cca == y_tst)
        
        print(f"fold: {i_fold+1} is complete")
        
    # Append the averaged fold accuracies for the subject
    LDA_acc_epoch.append(accuracy_epoch_lda.mean())
    LDA_acc_trial.append(accuracy_trial_lda.mean())
    CCA_acc_epoch.append(accuracy_epoch_cca.mean())
    CCA_acc_trial.append(accuracy_trial_cca.mean())
    
    # Append epoch predictions of all folds for subject
    epoch_predictions_saveall_lda[i] = epoch_predictions_save_lda
    epoch_predictions_saveall_cca[i] = epoch_predictions_save_cca

Subject:  sub-01
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-02
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-03
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-04
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-05
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-06
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-07
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-08
fold: 1 is complete
fold: 2 is complete
fold: 3 is complete
fold: 4 is complete
fold: 5 is complete
Subject:  sub-09
fold: 1 is complete
fold: 2 is complete
fold: 3

### Save data

In [10]:
"""
Save training history and epoch level classification
"""
# Save accuracies history
with open(os.path.join(path, 'history1', 'LDA_epoch_accuracies.txt'), 'w') as f:
    for acc in LDA_acc_epoch:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history1', 'LDA_trial_accuracies.txt'), 'w') as f:
    for acc in LDA_acc_trial:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history1', 'CCALDA_epoch_accuracies.txt'), 'w') as f:
    for acc in CCA_acc_epoch:
        f.write('%s\n' % acc)
with open(os.path.join(path, 'history1', 'CCALDA_trial_accuracies.txt'), 'w') as f:
    for acc in CCA_acc_trial:
        f.write('%s\n' % acc)
# Save epoch predictions of all fold of all subjects
np.save(os.path.join(path, 'history1', 'epoch_predictions_saveall_lda.npy'), epoch_predictions_saveall_lda)
np.save(os.path.join(path, 'history1', 'epoch_predictions_saveall_ldacca.npy'), epoch_predictions_saveall_cca)